# Create and run a local RAG pipeline from scratch

The goal of this notebook is to build a RAG (Retrieval Augmented Generation) pipeline from scratch and have it run on a local GPU.

Specifically, we'd like to be able to open a PDF file, ask questions (queries) of it and have them answered by a Large Language Model (LLM).

There are frameworks that replicate this kind of workflow, including LlamaIndex and LangChain, however, the goal of building from scratch is to be able to inspect and customize all the parts.

# What is RAG?

RAG stands for Retrieval Augmented Generation.

It was introduced in the paper  [Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks.](https://arxiv.org/abs/2005.11401)

Each step can be roughly broken down to:

Retrieval - Seeking relevant information from a source given a query. For example, getting relevant passages of Wikipedia text from a database given a question.    
Augmented - Using the relevant retrieved information to modify an input to a generative model (e.g. an LLM).    
Generation - Generating an output given an input. For example, in the case of an LLM, generating a passage of text given an input prompt.   